This code defines a convolutional neural network (CNN) to classify images of cats and dogs. The code is divided into the following sections:


### **1. Imports and Setup**

* **`%pip install tensorflow-gpu`:** Installs the TensorFlow library with GPU support. 
* **Imports:** Includes necessary libraries for image processing, model building, and visualization.
* **`BASE_DIR` and `DATA_DIR`:** Defines paths for the project root directory and the dataset location.
* **`image_width`, `image_height`, `batch_size`:** Defines constants for image dimensions and training batch size.
* **`print(tf.config.list_physical_devices('GPU'))`:** Prints available GPUs to ensure GPU acceleration is working.


In [ ]:
%pip install tensorflow-gpu

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import tensorflow as tf


from .preprocessing import Prc
from os.path import join
import pandas as pd
import os


BASE_DIR = '/content/'
DATA_DIR = '/content/dataset/'

image_width, image_height = 244, 244
batch_size = 50


print(tf.config.list_physical_devices('GPU'))

---

### **2.Preprocessing**

This section handles preprocessing of the dataset, including:

- Downloading the dataset
- Extracting the downloaded zip file
- Verifying the downloaded images
- Splitting the dataset into train and test sets

The `Prc` class is used to perform these operations. 

In [ ]:
prc = Prc()

prc.download_dataset(
    'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip',
    'catsVsdogs.zip'
)

prc.extract_zip(
    join(BASE_DIR, 'catsVsdogs.zip'),
    BASE_DIR
)

prc.image_verification(
    join(BASE_DIR, 'PetImages')
)

prc.split_dataset(
    join(BASE_DIR, 'PetImages'),
    DATA_DIR
)

---

### **3. Data Augmentation:**
This section defines data augmentation techniques to increase the dataset size and prevent overfitting.
  
  - ImageDataGenerator is used to apply transformations like zoom, flip, rotation, width/height shifting, and shear.
~

In [ ]:
train_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=50,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.4,
    fill_mode='nearest',
)

test_data_generator = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train = train_data_generator.flow_from_directory(join(BASE_DIR, 'Petimages', 'train'),
        class_mode='binary', batch_size=batch_size, target_size=(image_width, image_height))

test = test_data_generator.flow_from_directory(join(BASE_DIR, 'Petimages', 'test'),
       class_mode='binary', batch_size=batch_size, target_size=(image_width, image_height))

---

### **4. Model Definition:**

This section defines the CNN model using the Sequential API. 
  - The model consists of convolutional layers, pooling layers, flattening layer, dense layers, and dropout layers. 


In [ ]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

---

### **5. Callbacks:**

This section defines callbacks to enhance the training process:

- EarlyStopping: Stops training when the validation loss stops improving for a certain number of epochs.
- ReduceLROnPlateau: Reduces the learning rate when the validation loss plateaus.
- ModelCheckpoint: Saves the best performing model based on the validation loss.

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=10)
rlrop = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=0.001)

checkpoint = ModelCheckpoint('model.keras', monitor='val_loss', mode='min', save_best_only=True)

---

### **6. Model Training:**

This section trains the model using the train and test data generators, specifies the number of epochs, and applies the defined callbacks. 


In [ ]:
history = model.fit(
    train,
    validation_data=test,
    epochs=100,
    callbacks=[es, rlrop, checkpoint]
  )

---

### **7. Plotting Training History:**
This section plots the training and validation loss and accuracy over epochs.

In [ ]:
os.environ['KMP_DUPLICATE_LBT_OK'] = 'True'
pd.DataFrame(history.history).plot(figsize=(8, 5))

---

### **8. Saving the Model:**
This section saves the trained model

In [ ]:
output_path = join(BASE_DIR, 'models')
os.makedirs(output_path, exist_ok=True)

model.save(
    join(output_path, 'model.keras')
)